# <span style="color:red"> Lecture 14 - Creating/replacing variables.</span>

<font size = "5">

- Starting a new module on **manipulating data** in Pandas
- Will discuss adding new columns/variables to a DataFrame (data set)
- Will cover **cleaning** data and **recoding** variables

# <span style="color:red"> I. Import Libraries and Data </span>


<font size = "5">
Key libraries

In [1]:
import numpy as np
import pandas as pd

<font size = "5">

Read dataset of car racing circuits

- https://en.wikipedia.org/wiki/Formula_One <br>
- [See Data Source](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

In [2]:
circuits = pd.read_csv("data_raw/circuits.csv")

# <span style="color:red"> II. Examine Data </span>

<font size = "4">
Can view DataFrame using the DataWrangler extension on VS code. But information about the data can be accessed using Pandas commands 

In [3]:
# column/variable headings
print(circuits.columns.values)
print()
# number of rows and columns of DataFrame
print(circuits.shape)
print()
n_rows, n_cols = circuits.shape
print("# of rows:", n_rows)
print("# of columns:", n_cols)


['circuitId' 'circuitRef' 'name' 'location' 'country' 'lat' 'lng' 'alt'
 'url']

(77, 9)

# of rows: 77
# of columns: 9


<font size = "4">

We can also get the DataTypes for each column (dtypes)

In [4]:
print(circuits.dtypes)

circuitId       int64
circuitRef     object
name           object
location       object
country        object
lat           float64
lng           float64
alt            object
url            object
dtype: object


<font size = "4">

We've seen "circuits.apply", "circuits.shape", "circuits.dtypes", which are all **attributes** of the Pandas DataFrame we named "circuits"

The DataFrame also has **methods** (functions), like "circuits.apply" and "circuits.mean".

How do we remember all of these?

VS code can remind you of all available attributes and methods for an object. In the code cell below, type "circuits." (with the period) and then wait, without running the cell.

In [ ]:
# Type the following (including the period), then wait, without running the cell
# type this >> circuits.




<font size = "4">

**Note:** We've already seen how to add new columns/variables to a DataFrame. But we haven't seen how to rename the existing columns/variables. Some of the most straightforward are shown below

In [5]:
# Change the first and last column headings
print(circuits.columns.values)
print()

# Change last heading to "Uniform Resource Locator"
circuits.columns.values[-1] = "Uniform Resource Locator"

# Change first heading to "ID_No"
circuits.columns.values[0] = "ID_No"
print(circuits.columns.values)

['circuitId' 'circuitRef' 'name' 'location' 'country' 'lat' 'lng' 'alt'
 'url']

['ID_No' 'circuitRef' 'name' 'location' 'country' 'lat' 'lng' 'alt'
 'Uniform Resource Locator']


In [6]:
# Change back to "url" 

# Create a new data-frame and assign it to "circuits". 
# So we're overwriting the dataframe.
circuits = circuits.rename(columns={'Uniform Resource Locator': 'url'})

print(circuits.columns.values)

['ID_No' 'circuitRef' 'name' 'location' 'country' 'lat' 'lng' 'alt' 'url']


In [7]:
# Change back to "circuitID"

# Instead of making a new dataframe and over-writing the old one, change it "in-place"
circuits.rename(columns={'ID_No':'circuitID'}, inplace=True)

print(circuits.columns.values)

['circuitID' 'circuitRef' 'name' 'location' 'country' 'lat' 'lng' 'alt'
 'url']


# <span style="color:red"> III. Cleaning Data </span>

<font size = "5">

Data that should be numerical (e.g. altitude of a Formula One track) can have missing values or non-numerical replacements (like "N/A"). We want to manipulate the data to allow for numerical calculations.

In [8]:
print(circuits["lat"].mean())
print(circuits["lng"].mean())
print(circuits["alt"].mean())

33.442925064935054
1.0766831168831175


TypeError: Could not convert string '1018710913071322815310326444011621858357852235783745\N3837130266581460882227345432011261771456364841391263960948515333279139790214816788156762028470531581819129551851941616782-710825515\N\N' to numeric

In [9]:
# Let's try to figure out the problem
print(circuits.dtypes)

# The column "alt" has the DataType "object"

circuitID       int64
circuitRef     object
name           object
location       object
country        object
lat           float64
lng           float64
alt            object
url            object
dtype: object


In [10]:
print(circuits["alt"])
# these look like numbers, except the last two are "\N"

0      10
1      18
2       7
3     109
4     130
     ... 
72    108
73    255
74     15
75     \N
76     \N
Name: alt, Length: 77, dtype: object


<font size = "4">

- Each column of a Pandas DataFrame is a Pandas Series
- Each Pandas Series with "dtype: object" has a collection of methods known as "String Methods"
- To execute a method from this collection you must type: ``series_name.str.method_name``
- **NOTE**: Whenever I type something with a vague placeholder name like "series_name" or "method_name" this means that you should replace that word/phrase with an actual Python object or method/function name
- Within this collection, there is a method we want to use called ``.isnumeric``

In [11]:
# This is a Pandas Series
print(type(circuits["alt"])) 
print()
# This is a Pandas StringMethods object (collection of methods/functions)
print(type(circuits["alt"].str)) 
print()
# This is a method/function belonging to the StringMethods collection
print(type(circuits["alt"].str.isnumeric)) 

<class 'pandas.core.series.Series'>

<class 'pandas.core.strings.accessor.StringMethods'>

<class 'method'>


<font size = "4">

You'll get an error if you try to use ``.str`` with a column that doesn't have the "object" dtype

In [12]:
circuits["lat"].str

AttributeError: Can only use .str accessor with string values!

<font size = "4">

The ``series_name.str.isnumeric()`` method returns a Pandas Series with ``True`` everywhere the Series entry is deemed "numeric" and ``False`` otherwise.

In [13]:
print(circuits["alt"].str.isnumeric())

0      True
1      True
2      True
3      True
4      True
      ...  
72     True
73     True
74     True
75    False
76    False
Name: alt, Length: 77, dtype: bool


In [14]:
# We can reference subattributes of columns in ".query()"
non_numeric = circuits.query("  alt.str.isnumeric() == False ")
print(non_numeric)

    circuitID circuitRef                            name   location  \
22         80      vegas  Las Vegas Strip Street Circuit  Las Vegas   
71         73       baku               Baku City Circuit       Baku   
75         78     losail    Losail International Circuit  Al Daayen   
76         79      miami   Miami International Autodrome      Miami   

          country      lat       lng alt  \
22  United States  36.1147 -115.1730  \N   
71     Azerbaijan  40.3725   49.8533  -7   
75          Qatar  25.4900   51.4542  \N   
76            USA  25.9581  -80.2389  \N   

                                                  url  
22  https://en.wikipedia.org/wiki/Las_Vegas_Grand_...  
71     http://en.wikipedia.org/wiki/Baku_City_Circuit  
75  http://en.wikipedia.org/wiki/Losail_Internatio...  
76  http://en.wikipedia.org/wiki/Miami_Internation...  


<font size = "4">

Why is the string "-7" considered non-numeric?

In [15]:
# The pd.unique() function extracts unique values from a series/array

unique_non_numeric = pd.unique( non_numeric["alt"] )
print(unique_non_numeric)

['\\N' '-7']


<font size = "4">

We'll decide what we should replace "\N" and "-7" with

- "-7" is easy. We'll replace the string "-7" with the integer -7
- What about "\N"? It turns out the best choice is the "nan" (not a number) object from the Numpy library.

In [16]:
print(np.nan)

# ironically, the type of this object is a floating-point number...
print(type(np.nan))

nan
<class 'float'>


In [17]:
# unique_non_numeric is the list with ["\\N", "-7"]
# we'll make a list of the same length with the values we want to replace them with

replace_vals = [np.nan, -7]

# Overwrite the "alt" column, replacing every appearance from unique_non_numeric with
# the corresponding element of replace_vals
circuits["alt"] = circuits["alt"].replace(unique_non_numeric, replace_vals)

<font size = "4">

Did it work? Let's test it

In [18]:
non_numeric = circuits.query("  alt.str.isnumeric() == False  ")
print(non_numeric)

Empty DataFrame
Columns: [circuitID, circuitRef, name, location, country, lat, lng, alt, url]
Index: []


<font size = "4">

There are no rows with a non-numeric value for "alt"! Are we done? (No)

In [19]:
print(circuits["alt"].mean()) # get an error

TypeError: can only concatenate str (not "int") to str

<font size = "4">

What happened? We still don't have the right "dtype" to do calculations. We should have checked that in the first place

In [20]:
print(circuits.dtypes)

circuitID       int64
circuitRef     object
name           object
location       object
country        object
lat           float64
lng           float64
alt            object
url            object
dtype: object


<font size = "4">

The "alt" column still has the "object" dtype. But since we've cleaned the data, all of its elements **can be** converted to numeric values. The ``Pandas`` function ``to_numeric`` can be used to do this.

We'll create a new column to the DataFrame.

In [21]:
circuits["alt_numeric"] = pd.to_numeric(circuits["alt"])
print(circuits["alt_numeric"].mean())

248.1891891891892


In [22]:
# After the cleaning process is done, you may want to store the dataset again
# It's recommended to do this in a separate file from the original
# That way you can always go back to the original if you made a programming error

circuits.to_csv("data_clean/circuits.csv")

<font size = "4">

**Exercise**: When I was creating this notebook, I had the following question. "If I read in the .csv file I just created and save it as a new DataFrame, will I have to re-do any part of the cleaning process?" Test this out for yourself.

**Exercise**: Read in the .csv file that was just created, saving it as a new DataFrame. You'll see there is an extra column/variable called "Unnamed" with the values 0, 1, 2, ..., 75, 76. Type ``help(circuits.to_csv)`` to figure out how to create a new .csv file without this extra column.

In [29]:
# your answer here
new_df = pd.read_csv("data_clean/circuits.csv")

# First, tried line below, got an error
# non_num= new_df.query("  alt.str.isnumeric() == False  ")

print(new_df["alt"].dtype) 
# Since we cleaned this column, Pandas recognized all elements
# were numerical and changed the dtype!!!

print(new_df["alt"].mean())

# So no, don't need to do any re-cleaning. 
# Plus, "alt_numeric" turns out to be unnecessary.

float64
248.1891891891892


In [31]:
# your answer here
circuits.to_csv("data_clean/circuits_no_idx.csv", index = False)

<font size = "4">

**Exercise:**

- Use ".replace()" with the "country" column
- Replace "USA" with "United States"

In [32]:
# Write your own code
circuits["country"] = circuits["country"].replace("USA", "United States")


# <span style="color:red"> IV. Recoding Variables</span>


<font size = "4">

"Recoding" variables means changing/re-expressing existing variables into a new set of values that may be more useful for data analysis.

Examples:

- A variable called "age" consists of integers beginning at 0 (how old the person is in terms of years). We can transform this into a categorical variable with the values "child", "teenager", "young adult", "adult", "senior"

- A variable called "submission_time" records the time that a student submitted an assignment to Gradescope. We can transform this into a categorical variable with the values "on time" and "late"

- A variable called "candidate_preference" is taken from a survey of people asking which of two presidential candidates they will vote for. This is a categorical variable with the values "Candidate A", "Candidate B", "Don't Know", "Refused", "Blank", "Other Candidate". This is recoded to another categorical variable with only 3 values "Candidate A", "Candidate B", and "Neither".

- We convert a variable "height" measured in inches, to a corresponding variable measuring height in centimeters.

- A variable called "lat" consists of floating point numbers consisting of a geographical coordinate measuring the distance north or south of the Equator. We change this into a categorical variable with the values "north" or "south".

We'll perform this last one in 3 different ways. In each way, we will consider as "south" those circuits with $-90 < \textrm{lat} \leq 0$ and "north" as circuits with $0 < \textrm{lat} \leq 90$

In [ ]:
nrows, ncols = circuits.shape 

col_no = 5 # column 5 is "lat"
hemisphere = []

for k in range(nrows):
    lat_val = circuits.iloc[k, col_no]
    if lat_val <= 0:
        hemisphere.append("South")
    else:
        hemisphere.append("North")

# could overwrite "lat", but is almost always better to keep the original variable
circuits["hemisphere"] = hemisphere

# Note: I made sure the data in this column didn't need to be cleaned first!!

In [ ]:
# reset the column to verify the next cell actually works
circuits["hemisphere"] = "N/A"

In [ ]:
def check_hemisphere(lat):
    if lat_val <= 0:
        return "South" 
    else:
        return "North"

circuits["hemisphere"] = circuits["lat"].apply(check_hemisphere)

# Q: What error did I make?


In [ ]:
# reset the column to verify the next cell actually works
circuits["hemisphere"] = "N/A"

In [ ]:
# Use the Pandas function called ".cut", which you can use to segment data
# into intervals or "bins" 

lat_bins = [-90, 0, 90]
# lat_bins = [float("-inf"), 0, float("inf")]

lat_labels = ["South", "North"]
circuits["hemisphere"] = pd.cut(circuits["lat"], 
                                bins = lat_bins,
                                right = True,
                                labels = lat_labels)


# Note: if we set lat_bins = [float("-inf"), 0, float("inf")]
# then intervals are "Less than or equal to 0" and "Above 0"
# float("inf") and float("-inf") represent positive infinity and negative infinity
# The "right" command indicates that the right interval is *inclusive*,
# i.e. "less than or equal to"
                            


<font size = "4">

**Exercise:** Recode the altitude variable into a categorial variable with the values

- "Low altitude" if $\textrm{altitude} \leq 45$
- "Medium altitude" if $45 < \textrm{altitude} \leq 145$
- "High altitude" if $145 < \textrm{altitude}$

**Hint**: Should you use the original altitude column or the one we cleaned?

In [ ]:
# your answer here
alt_bins = [float("-inf"), 45, 145, float("inf")]

alt_labels = ["Low altitude", "Medium altitude", "High altitude"]
circuits["alt_type"] = pd.cut(circuits["alt_numeric"], 
                                bins = alt_bins,
                                right = True,
                                labels = alt_labels)


<font size = "4">

**Exercise:** What happens if you use the ``Pandas`` function ``.cut`` if the bins you provide don't capture all the data in the column? What happens if there are missing values in the column? Test it out by recoding the altitude

- "Group A" if $0 < \textrm{altitude} \leq 120$
- "Group B" if $120 < \textrm{altitude} \leq 400$
- "Group C" if $400 < \textrm{altitude} \leq 800$

In [ ]:
# Write your own code
group_bins = [0, 120, 400, 800]

group_labels = ["Group A", "Group B", "Group C"]
circuits["alt_group"] = pd.cut(circuits["alt_numeric"], 
                                bins = group_bins,
                                right = True,
                                labels = group_labels)


alt_over_800 = circuits.query("alt_numeric > 800")
print(alt_over_800["alt_group"])
# Looks like "np.nan" gets entered

29    NaN
31    NaN
35    NaN
Name: alt_group, dtype: category
Categories (3, object): ['Group A' < 'Group B' < 'Group C']
